## Preprocessing

In [18]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [19]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
#  YOUR CODE GOES HERE
application_df = application_df.drop(columns=['EIN', 'NAME'])

application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [20]:
# Determine the number of unique values in each column.
#  YOUR CODE GOES HERE
unique_values = application_df.nunique()

# Display the unique values for each column
print(unique_values)

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64


In [21]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
#  YOUR CODE GOES HERE
application_type_counts = application_df['APPLICATION_TYPE'].value_counts()

# Display the value counts
print(application_type_counts)

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64


In [22]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
#  YOUR CODE GOES HERE
cutoff_value = 500

# Create a list of application types to replace
application_types_to_replace = application_type_counts[application_type_counts < cutoff_value].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [23]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
#  YOUR CODE GOES HERE
classification_counts = application_df['CLASSIFICATION'].value_counts()

# Display the classification counts
print(classification_counts)

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64


In [24]:
# You may find it helpful to look at CLASSIFICATION value counts >1
#  YOUR CODE GOES HERE
classification_counts_filtered = classification_counts[classification_counts > 1]

# Display the filtered classification counts
print(classification_counts_filtered)

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64


In [25]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
#  YOUR CODE GOES HERE
cutoff_value = 100

# Create a list of classifications to replace
classifications_to_replace = classification_counts[classification_counts < cutoff_value].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
Other      669
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
Name: count, dtype: int64

In [26]:
# Convert categorical data to numeric with `pd.get_dummies`
#  YOUR CODE GOES HERE

# Check data types and number of unique values
categorical_candidates = application_df.select_dtypes(include=['object']).columns.tolist()

# Add columns with few unique values (e.g., less than 10 unique values) to the list
additional_categorical = application_df.nunique()[application_df.nunique() < 10].index.tolist()

# Combine the two lists
categorical_columns = list(set(categorical_candidates + additional_categorical))

# Display the categorical columns
print("Potential categorical variables:", categorical_columns)

application_df = pd.get_dummies(application_df, columns=[
    'APPLICATION_TYPE', 
    'INCOME_AMT', 
    'AFFILIATION', 
    'USE_CASE', 
    'CLASSIFICATION', 
    'IS_SUCCESSFUL', 
    'STATUS', 
    'ORGANIZATION', 
    'SPECIAL_CONSIDERATIONS'
], drop_first=True)

# Display the first few rows of the updated DataFrame
application_df.head()

Potential categorical variables: ['SPECIAL_CONSIDERATIONS', 'STATUS', 'CLASSIFICATION', 'IS_SUCCESSFUL', 'USE_CASE', 'AFFILIATION', 'INCOME_AMT', 'ORGANIZATION', 'APPLICATION_TYPE']


,ASK_AMT,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,INCOME_AMT_1-9999,...,CLASSIFICATION_C4000,CLASSIFICATION_C5000,CLASSIFICATION_C7000,CLASSIFICATION_Other,IS_SUCCESSFUL_1,STATUS_1,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust,SPECIAL_CONSIDERATIONS_Y
0,5000,True,False,False,False,False,False,False,False,False,...,False,False,False,False,True,True,False,False,False,False
1,108590,False,False,True,False,False,False,False,False,True,...,False,False,False,False,True,True,True,False,False,False
2,5000,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
3,6692,False,False,True,False,False,False,False,False,False,...,False,False,False,False,True,True,False,False,True,False
4,142590,False,False,True,False,False,False,False,False,False,...,False,False,False,False,True,True,False,False,True,False


In [27]:
# Split our preprocessed data into our features and target arrays
#  YOUR CODE GOES HERE

# Define the target variable (y)
y = application_df['IS_SUCCESSFUL_1']

# Define the features (X) by dropping the target column
X = application_df.drop(columns=['IS_SUCCESSFUL_1'])

# Display the shapes of X and y to confirm the split
print(f"Features shape: {X.shape}")
print(f"Target shape: {y.shape}")

# Split the preprocessed data into a training and testing dataset
#  YOUR CODE GOES HERE
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Display the shapes of the training and testing datasets
print("Training features shape:", X_train.shape)
print("Testing features shape:", X_test.shape)
print("Training target shape:", y_train.shape)
print("Testing target shape:", y_test.shape)

Features shape: (34299, 42)
Target shape: (34299,)
Training features shape: (27439, 42)
Testing features shape: (6860, 42)
Training target shape: (27439,)
Testing target shape: (6860,)


In [28]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

print("Scaled training features shape:", X_train_scaled.shape)
print("Scaled testing features shape:", X_test_scaled.shape)

Scaled training features shape: (27439, 42)
Scaled testing features shape: (6860, 42)


## Compile, Train and Evaluate the Model

In [65]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE

nn = tf.keras.models.Sequential()

# First hidden layer
#  YOUR CODE GOES HERE
nn.add(tf.keras.layers.Dense(units=128, activation='relu', input_dim=X_train_scaled.shape[1]))
nn.add(tf.keras.layers.Dropout(0.1))

# Second hidden layer
#  YOUR CODE GOES HERE
nn.add(tf.keras.layers.Dense(units=64, activation='relu'))
nn.add(tf.keras.layers.Dropout(0.1))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=32, activation='relu'))

# Output layer
#  YOUR CODE GOES HERE
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_18 (Dense)                │ (None, 128)            │         5,504 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_20 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_21 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 15,873 (62.00 KB)

 Trainable params: 15,873 (62.00 KB)

 Non-trainable params: 0 (0.00 B)

In [66]:
# Compile the model
#  YOUR CODE GOES HERE
optimizer = tf.keras.optimizers.Adam(learning_rate=0.00001)
nn.compile(optimizer=optimizer, 
           loss='binary_crossentropy', 
           metrics=['accuracy'])

In [67]:
# Train the model
#  YOUR CODE GOES HERE
history = nn.fit(X_train_scaled, y_train, 
                 epochs=150, 
                 batch_size=64, 
                 validation_split=0.2, 
                 verbose=1)

Epoch 1/150
343/343 ━━━━━━━━━━━━━━━━━━━━ 1s 837us/step - accuracy: 0.5648 - loss: 0.6929 - val_accuracy: 0.6421 - val_loss: 0.6759
Epoch 2/150
343/343 ━━━━━━━━━━━━━━━━━━━━ 0s 577us/step - accuracy: 0.6112 - loss: 0.6740 - val_accuracy: 0.6880 - val_loss: 0.6587
Epoch 3/150
343/343 ━━━━━━━━━━━━━━━━━━━━ 0s 595us/step - accuracy: 0.6496 - loss: 0.6614 - val_accuracy: 0.7003 - val_loss: 0.6440
Epoch 4/150
343/343 ━━━━━━━━━━━━━━━━━━━━ 0s 613us/step - accuracy: 0.6806 - loss: 0.6481 - val_accuracy: 0.7085 - val_loss: 0.6310
Epoch 5/150
343/343 ━━━━━━━━━━━━━━━━━━━━ 0s 587us/step - accuracy: 0.6850 - loss: 0.6367 - val_accuracy: 0.7181 - val_loss: 0.6193
Epoch 6/150
343/343 ━━━━━━━━━━━━━━━━━━━━ 0s 624us/step - accuracy: 0.6960 - loss: 0.6290 - val_accuracy: 0.7218 - val_loss: 0.6089
Epoch 7/150
343/343 ━━━━━━━━━━━━━━━━━━━━ 0s 569us/step - accuracy: 0.7001 - loss: 0.6221 - val_accuracy: 0.7230 - val_loss: 0.5995
Epoch 8/150
343/343 ━━━━━━━━━━━━━━━━━━━━ 0s 576us/step - accuracy: 0.7066 - loss: 0

In [68]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 0s - 306us/step - accuracy: 0.7246 - loss: 0.5550
Loss: 0.5550450086593628, Accuracy: 0.7246355414390564


In [16]:
# Export our model to HDF5 file
#  YOUR CODE GOES HERE
nn.save('trained_model.h5')